## 마크다운으로 작성된 코드를 html로 변환하고자 함

1. re 정규식을 이용하여 처리.

In [5]:
import re

def markdown_to_html(markdown_text):
    """Converts markdown text to HTML, including tables."""
    # 1. 테이블부터 전처리
    html_text = markdown_table_to_html(markdown_text)
    
    # 2. 헤더 설정
    for i in range(5, 0, -1):
        html_text = re.sub(f'^{"#" * i} (.+)$', f'<h{i+1}>\\1</h{i+1}>', html_text, flags=re.MULTILINE)
    
    # 3. 나머지 마크다운 문법 처리
    html_text = re.sub(r'^\* (.+)$', r'<li>\1</li>', html_text, flags=re.MULTILINE)
    html_text = re.sub(r'(?<!<ul>)\s*<li>(.+?)</li>\s*(?!</ul>)', r'<ul><li>\1</li></ul>', html_text, flags=re.DOTALL)
    html_text = re.sub(r'</ul>\s*<ul>', '', html_text)
    html_text = re.sub(r'\*\*(.+?)\*\*', r'<b>\1</b>', html_text)
    html_text = re.sub(r'\*(.+?)\*', r'<i>\1</i>', html_text)
    
    # \n 개행문자(?)를 <br> 태그로 변경하여 줄바꿈
    html_text = html_text.replace('\n\n', '<br><br>')
    html_text = html_text.replace('\n', '<br>')
    
    return html_text

def markdown_table_to_html(markdown_text):
    """마크다운 테이블 형식을 html코드로 변환"""
    def convert_table(match):
        table = match.group(0)
        rows = table.strip().split('\n')
        header_row = rows[0].strip().split('|')[1:-1]
        separator_row = rows[1].strip().split('|')[1:-1]
        data_rows = [row.strip().split('|')[1:-1] for row in rows[2:]]
        
        html_table = '<table><thead><tr>'
        for cell in header_row:
            html_table += f'<th>{cell.strip()}</th>'
        html_table += '</tr></thead><tbody>'
        for row in data_rows:
            html_table += '<tr>'
            for cell in row:
                html_table += f'<td>{cell.strip()}</td>'
            html_table += '</tr>'
        html_table += '</tbody></table>'
        return html_table

    # 데이터 프레임 찾은 후 처리
    pattern = r'\|(.+\|)+\n\|(\s*:?-+:?\s*\|)+\n(\|.+\|)+'
    html_text = re.sub(pattern, convert_table, markdown_text, flags=re.MULTILINE)
    return html_text

# 예시
markdown_text = """
## 주택도시기금 디딤돌대출

### **1. 개요**

**주택도시기금 디딤돌대출**은 부부합산 연소득이 일정 금액 이하인 무주택 세대주를 대상으로 주택 구입 자금을 융자 지원하는 제도입니다. 

### **2. 지원 내용**

#### **2.1 지원 대상**

* 대출 신청인과 배우자의 합산 총소득이 연간 6천만 원 이하인 무주택 세대주
    * 생애 최초 주택구입자는 연간 7천만 원, 신혼가구는 연간 8.5천만 원 이하까지 지원
    * 유한책임방식 디딤돌대출 이용 시 부부 연소득 조건 상이 (3천만원~6천만원)
* 만 30세 미만 단독세대주는 제외 (단, 상속으로 인한 단독세대주 등 예외 경우 존재)
* 구매 용도로만 가능 (상환, 보전 용도 불가)
* 대출 실행 후 1개월 이내 전입, 1년 이상 실거주 필수 (단, 타당한 사유 시 연장 가능)

#### **2.2 지원 내용**

* **대출 한도**: 
    * 호당 2.5억 원 이내
    * 신혼가구, 2자녀 이상 가구: 4억 원 이내
    * 생애 최초 주택 구입자: 3억 원 이내
* **대출 금리**: 연 2.45% ~ 3.55% (소득 및 만기에 따라 차등 적용)
* **금리 우대**: 
    * 다자녀 가구 0.7%p, 2자녀 가구 0.5%p, 1자녀 가구 0.3%p
    * 연소득 6천만 원 이하 한부모 가구 0.5%p
    * 다문화 가구, 장애인 가구, 생애 최초 주택구입자, 신혼가구(결혼예정자 포함) 각각 0.2%p
        * (단, 중복 적용 불가, 다자녀 우대 금리는 상기 타 우대 금리와 중복 적용 가능)
    * 본인 또는 배우자 명의의 청약(종합)저축 가입 중인 경우 0.3~0.7%p 금리 우대
    * 국토교통부 전자계약시스템 활용 매매계약 체결 시 0.1%p 우대 (2023.12.31 신규 접수분까지 한시적 운영) 
        * 청약저축, 부동산 전자계약 우대금리는 타 우대금리와 중복 적용 가능
    * 우대금리 적용 결과 최종 대출금리가 1.5%p 미만인 경우에는 1.5%p 적용
    * 생애 최초 주택구입 연소득 7천만원 이하 신혼가구 최대 0.3%p 추가 인하 (하한선 연 1.2%)

### **3. 신청 방법**

* **신청 기간**: 접수 기관 별 상이
* **신청 방법**: 
    * 한국주택금융공사 또는 기금e든든 웹사이트 신청
    * 수탁은행(우리, 국민, 기업, 농협, 신한) 방문 신청
* **제출 서류**: 소득 및 재직 증빙 서류, 개인 정보 제공 동의서 등

### **4. 문의처**

* 국토교통부 (☎1599-0001)
* 주택도시보증공사 (☎1566-9009)
* 한국주택금융공사 (☎1688-8114)
* 우리은행 (☎1599-0800)
* 국민은행 (☎1599-1771)
* 기업은행 (☎1566-2566)
* 농협은행 (☎1588-2100)
* 신한은행 (☎1599-8000)
"""

html_text = markdown_to_html(markdown_text)
print(html_text)

<br><h3>주택도시기금 디딤돌대출</h3><br><br><h4><b>1. 개요</b></h4><br><br><b>주택도시기금 디딤돌대출</b>은 부부합산 연소득이 일정 금액 이하인 무주택 세대주를 대상으로 주택 구입 자금을 융자 지원하는 제도입니다. <br><br><h4><b>2. 지원 내용</b></h4><br><br><h5><b>2.1 지원 대상</b></h5><ul><li>대출 신청인과 배우자의 합산 총소득이 연간 6천만 원 이하인 무주택 세대주</li></ul>* 생애 최초 주택구입자는 연간 7천만 원, 신혼가구는 연간 8.5천만 원 이하까지 지원<br>    <i> 유한책임방식 디딤돌대출 이용 시 부부 연소득 조건 상이 (3천만원~6천만원)<ul><li>만 30세 미만 단독세대주는 제외 (단, 상속으로 인한 단독세대주 등 예외 경우 존재)</li><li>구매 용도로만 가능 (상환, 보전 용도 불가)</li><li>대출 실행 후 1개월 이내 전입, 1년 이상 실거주 필수 (단, 타당한 사유 시 연장 가능)</li></ul><h5><b>2.2 지원 내용</b></h5><ul><li><b>대출 한도</b>: </li></ul></i> 호당 2.5억 원 이내<br>    * 신혼가구, 2자녀 이상 가구: 4억 원 이내<br>    <i> 생애 최초 주택 구입자: 3억 원 이내<ul><li><b>대출 금리</b>: 연 2.45% ~ 3.55% (소득 및 만기에 따라 차등 적용)</li><li><b>금리 우대</b>: </li></ul></i> 다자녀 가구 0.7%p, 2자녀 가구 0.5%p, 1자녀 가구 0.3%p<br>    * 연소득 6천만 원 이하 한부모 가구 0.5%p<br>    * 다문화 가구, 장애인 가구, 생애 최초 주택구입자, 신혼가구(결혼예정자 포함) 각각 0.2%p<br>        * (단, 중복 적용 불가, 다자녀 우대 금리는 상기 타 우대 금리와 중복 적용 가능)<br>    * 본인 또는 배우자 명의의 청약(종합)저축 가입 중인